In [21]:
import os

versions = ['HU33', 'SC06', 'FI66', 'FI80']
chroma_type = ['stft', 'cq', 'cens']

template_type = ['binary', 'ks', 'harmonic']

key2num = {
    "A:maj": 0, "A#:maj": 1, "Bb:maj": 1, "B:maj": 2,
    "C:maj": 3, "C#:maj": 4, "Db:maj": 4, "D:maj": 5, "D#:maj": 6, "Eb:maj": 6, 
    "E:maj": 7, "F:maj": 8, "F#:maj": 9, "G:maj": 10, "G#:maj": 11, "Ab:maj": 11,
    "A:min": 12, "A#:min": 13, "Bb:min": 13, "B:min": 14,
    "C:min": 15, "C#:min": 16, "Db:min": 16, "D:min": 17, "D#:min": 18, "Eb:min": 18, 
    # There is D#:min and Eb:min, This could be a problem when finding key
    "E:min": 19, "F:min": 20, "F#:min": 21, "G:min": 22, "G#:min": 23, "Ab:min": 23,
    }

num2key = {
    0: "A:maj", 1: "A#:maj", 2: "B:maj", 3: "C:maj", 4: "C#:maj", 5: "D:maj", 6: "D#:maj",
    7: "E:maj", 8:"F:maj", 9: "F#maj", 10: "G:maj", 11: "G#:maj", 
    12: "A:min", 13: "A#:min", 14: "B:min", 15: "C:min", 16: "C#:min", 17: "D:min", 18: "D#min",
    19: "E:min", 20: "F:min", 21: "F#:min", 22: "G:min", 23: "G#:min"
}

In [17]:
predict = []

key_path = "result/SWD/local_predict/FI66/1/cens/binary.txt"
with open(key_path, mode='r') as f:
    
    
    pre_frame_i = 10
    frame_i = 0

    pre_key = int(f.readline())

    while True:
        # Parse the string
        
        line = f.readline()
        if not line:
            break
        
        if frame_i < 10:
            frame_i += 1
            continue
        
        
        key = int(line)
        
        if key != pre_key:
            
            predict.append([pre_frame_i, frame_i, pre_key])
            pre_key = key
            pre_frame_i = frame_i
        
        frame_i += 1
    predict.append([pre_frame_i, frame_i, pre_key])

In [18]:
predict

[[10, 311, 15],
 [311, 549, 6],
 [549, 1048, 15],
 [1048, 1334, 6],
 [1334, 1834, 15],
 [1834, 1966, 6],
 [1966, 2439, 15],
 [2439, 3044, 3],
 [3044, 3343, 22]]

In [31]:
key_path = "result/SWD/local_predict_csv/FI66/1/cens/binary.csv"
os.makedirs(os.path.dirname(key_path), exist_ok=True)
with open(key_path, mode='w') as f:
    f.write('start;end;key\n')
    for p in predict:
        key = num2key[p[2]]
        f.write(str(p[0]/10) + ';' + str(p[1]/10) + ';"' + str(key) + '"\n')

        


In [38]:
import mir_eval

(ref_intervals, ref_labels) = mir_eval.io.load_labeled_intervals('result/SWD/local_predict_csv/FI66/1/cens/binary.csv', 
                                                                 delimiter=';',
                                                                 comment='start;end;key')
(ref_intervals2, ref_labels2) = mir_eval.io.load_labeled_intervals('data/SWD/02_Annotations/ann_audio_localkey-ann1/Schubert_D911-01_HU33.csv', 
                                                                 delimiter=';',
                                                                 comment='start;end;key')


In [40]:
mir_eval.chord.underseg(ref_intervals, ref_intervals2)

0.8597332632547138

In [34]:
ref_intervals

array([[  1. ,  31.1],
       [ 31.1,  54.9],
       [ 54.9, 104.8],
       [104.8, 133.4],
       [133.4, 183.4],
       [183.4, 196.6],
       [196.6, 243.9],
       [243.9, 304.4],
       [304.4, 334.3]])

In [39]:
ref_intervals2

array([[  1.26,  31.68],
       [ 31.68,  40.56],
       [ 40.56,  49.  ],
       [ 49.  ,  98.92],
       [ 98.92, 107.8 ],
       [107.8 , 116.24],
       [116.24, 147.86],
       [147.86, 166.52],
       [166.52, 175.44],
       [175.44, 184.  ],
       [184.  , 217.16],
       [217.16, 239.62],
       [239.62, 250.  ],
       [250.  , 283.28],
       [283.28, 305.68]])